# Several tests

## repository/

In [ ]:
%%writefile CMakeLists.txt
# Define the minimum required CMake version. Version 3.17 is specified in the parent CMakeLists.txt
# for FindCUDAToolkit support, so we keep this.
cmake_minimum_required(VERSION 3.17)

# Define the C++ standard for compilation. Although the examples are Fortran, the C++ library
# they link against uses this standard.
set(CMAKE_CXX_STANDARD 17)

# Define the default build type if not specified by the user.
if (NOT CMAKE_BUILD_TYPE)
  set(CMAKE_BUILD_TYPE RelWithDebInfo)
endif()

# Apply a CMake policy. Kept from the parent CMakeLists.txt.
cmake_policy(SET CMP0057 NEW)

# --- User-Defined Build Options (from parent CMakeLists.txt) ---
# We only keep the options relevant to the Fortran examples and their dependencies.
# TORCHFORT_CUDA_CC_LIST: List of CUDA compute capabilities. Used to generate CUF_GPU_ARG.
set(TORCHFORT_CUDA_CC_LIST "70;80;90" CACHE STRING "List of CUDA compute capabilities for GPU compilation.")
# TORCHFORT_NCCL_ROOT: Path to find the NCCL installation. Required if using distributed MPI with GPU.
set(TORCHFORT_NCCL_ROOT CACHE STRING "Path to search for NCCL installation.")
# TORCHFORT_YAML_CPP_ROOT: Path to find the yaml-cpp installation. The C++ lib links against it.
set(TORCHFORT_YAML_CPP_ROOT CACHE STRING "Path to search for yaml-cpp installation.")
# TORCHFORT_ENABLE_GPU: Enable GPU/CUDA support. Affects CUDA/NCCL search and compilation options.
option(TORCHFORT_ENABLE_GPU "Enable GPU/CUDA support" ON)
# Options like BUILD_FORTRAN, BUILD_EXAMPLES, BUILD_TESTS are removed as this file ALREADY builds the Fortran examples.

# Check if TORCHFORT_YAML_CPP_ROOT was defined, otherwise show fatal error (kept from parent).
if (NOT TORCHFORT_YAML_CPP_ROOT)
  message(FATAL_ERROR "Please set TORCHFORT_YAML_CPP_ROOT to yaml-cpp installation directory.")
endif()

# Define the project languages. We need Fortran for the examples and CXX for the library they link to.
project(CombinedFortranExamples LANGUAGES Fortran CXX) # Project name changed to reflect content

# --- External Dependency Search (from parent CMakeLists.txt and Fortran example) ---

# MPI: Required for distributed examples.
find_package(MPI REQUIRED)

# HDF5: Required for data reading/writing in examples.
find_package(HDF5 COMPONENTS Fortran REQUIRED)

# CUDA and NCCL: Required if GPU support is enabled.
if (TORCHFORT_ENABLE_GPU)
  find_package(CUDAToolkit REQUIRED)

  # Search for NVHPC compiler and CMake configurations (from parent).
  find_program(NVHPC_CXX_BIN "nvc++")
  if (NVHPC_CXX_BIN)
    string(REPLACE "compilers/bin/nvc++" "cmake" NVHPC_CMAKE_DIR ${NVHPC_CXX_BIN})
    # Add the NVHPC CMake directory to CMAKE_PREFIX_PATH to help find other NVHPC packages.
    set(CMAKE_PREFIX_PATH "${CMAKE_PREFIX_PATH};${NVHPC_CMAKE_DIR}")
    find_package(NVHPC COMPONENTS "") # Find the main NVHPC package.
  endif()

  # Search for the NCCL library (from parent). Uses TORCHFORT_NCCL_ROOT or NVHPC configurations.
  if (TORCHFORT_NCCL_ROOT)
    find_path(NCCL_INCLUDE_DIR REQUIRED
      NAMES nccl.h
      HINTS ${TORCHFORT_NCCL_ROOT}/include
    )
    find_library(NCCL_LIBRARY REQUIRED
      NAMES nccl
      HINTS ${TORCHFORT_NCCL_ROOT}/lib
    )
  else()
    if (NVHPC_FOUND)
      find_package(NVHPC REQUIRED COMPONENTS NCCL) # Try to find NCCL via NVHPC.
      find_library(NCCL_LIBRARY
        NAMES nccl
        HINTS ${NVHPC_NCCL_LIBRARY_DIR} # NCCL directory provided by NVHPC.
      )
      # Derive the include directory from the NVHPC NCCL library directory.
      string(REPLACE "/lib" "/include" NCCL_INCLUDE_DIR ${NVHPC_NCCL_LIBRARY_DIR})
    else()
      message(FATAL_ERROR "Cannot find NCCL library. Please set TORCHFORT_NCCL_ROOT or ensure NVHPC is configured.")
    endif()
  endif()
  message(STATUS "Using NCCL library: ${NCCL_LIBRARY}")
endif()

# PyTorch: The internal library links against PyTorch/LibTorch.
find_package(Torch REQUIRED)

# yaml-cpp: The internal library links against yaml-cpp.
# Use TORCHFORT_YAML_CPP_ROOT to find.
find_path(YAML_CPP_INCLUDE_DIR REQUIRED
  NAMES yaml-cpp/yaml.h
  HINTS ${TORCHFORT_YAML_CPP_ROOT}/include
)
find_library(YAML_CPP_LIBRARY REQUIRED
  NAMES yaml-cpp
  HINTS ${TORCHFORT_YAML_CPP_ROOT}/lib
)
message(STATUS "Using yaml-cpp library: ${YAML_CPP_LIBRARY}")

# --- Internal C++ Library Definition (from parent CMakeLists.txt) ---
# The Fortran examples link against this library. We include all C++ sources
# listed in the parent to ensure the library is built correctly.
add_library(torchfort SHARED) # Keep the original target name that examples expect.
set_target_properties(torchfort PROPERTIES LIBRARY_OUTPUT_DIRECTORY ${CMAKE_BINARY_DIR}/lib)

# List of C++ source files (copy the necessary .cpp files to src/csrc/ relative to this CMakeLists.txt)
target_sources(torchfort
  PRIVATE
  ${CMAKE_CURRENT_SOURCE_DIR}/src/csrc/distributed.cpp
  ${CMAKE_CURRENT_SOURCE_DIR}/src/csrc/logging.cpp
  ${CMAKE_CURRENT_SOURCE_DIR}/src/csrc/model_state.cpp
  ${CMAKE_CURRENT_SOURCE_DIR}/src/csrc/model_wrapper.cpp
  ${CMAKE_CURRENT_SOURCE_DIR}/src/csrc/model_pack.cpp
  ${CMAKE_CURRENT_SOURCE_DIR}/src/csrc/param_map.cpp
  ${CMAKE_CURRENT_SOURCE_DIR}/src/csrc/setup.cpp
  ${CMAKE_CURRENT_SOURCE_DIR}/src/csrc/torchfort.cpp
  ${CMAKE_CURRENT_SOURCE_DIR}/src/csrc/training.cpp
  ${CMAKE_CURRENT_SOURCE_DIR}/src/csrc/utils.cpp
  ${CMAKE_CURRENT_SOURCE_DIR}/src/csrc/losses/l1_loss.cpp
  ${CMAKE_CURRENT_SOURCE_DIR}/src/csrc/losses/mse_loss.cpp
  ${CMAKE_CURRENT_SOURCE_DIR}/src/csrc/losses/torchscript_loss.cpp
  ${CMAKE_CURRENT_SOURCE_DIR}/src/csrc/lr_schedulers/cosine_annealing_lr.cpp
  ${CMAKE_CURRENT_SOURCE_DIR}/src/csrc/lr_schedulers/multistep_lr.cpp
  ${CMAKE_CURRENT_SOURCE_DIR}/src/csrc/lr_schedulers/polynomial_lr.cpp
  ${CMAKE_CURRENT_SOURCE_DIR}/src/csrc/lr_schedulers/scheduler_setup.cpp
  ${CMAKE_CURRENT_SOURCE_DIR}/src/csrc/lr_schedulers/step_lr.cpp
  ${CMAKE_CURRENT_SOURCE_DIR}/src/csrc/lr_schedulers/linear_lr.cpp
  ${CMAKE_CURRENT_SOURCE_DIR}/src/csrc/models/mlp_model.cpp
  ${CMAKE_CURRENT_SOURCE_DIR}/src/csrc/models/sac_model.cpp
  ${CMAKE_CURRENT_SOURCE_DIR}/src/csrc/models/actor_critic_model.cpp
  ${CMAKE_CURRENT_SOURCE_DIR}/src/csrc/rl/policy.cpp
  ${CMAKE_CURRENT_SOURCE_DIR}/src/csrc/rl/utils.cpp
  ${CMAKE_CURRENT_SOURCE_DIR}/src/csrc/rl/off_policy/interface.cpp
  ${CMAKE_CURRENT_SOURCE_DIR}/src/csrc/rl/off_policy/ddpg.cpp
  ${CMAKE_CURRENT_SOURCE_DIR}/src/csrc/rl/off_policy/td3.cpp
  ${CMAKE_CURRENT_SOURCE_DIR}/src/csrc/rl/off_policy/sac.cpp
  ${CMAKE_CURRENT_SOURCE_DIR}/src/csrc/rl/on_policy/interface.cpp
  ${CMAKE_CURRENT_SOURCE_DIR}/src/csrc/rl/on_policy/ppo.cpp
)

# Include directories for the C++ library (from parent).
target_include_directories(torchfort
  PRIVATE
  ${CMAKE_CURRENT_SOURCE_DIR}/src/csrc/include # Include the lib's own headers
  ${YAML_CPP_INCLUDE_DIR} # Include yaml-cpp headers
  ${MPI_CXX_INCLUDE_DIRS} # Include MPI C++ headers
  ${TORCH_INCLUDE_DIRS} # Include PyTorch headers
)
if (TORCHFORT_ENABLE_GPU)
  target_include_directories(torchfort
    PRIVATE
    ${CUDAToolkit_INCLUDE_DIRS} # Include CUDA Toolkit headers
    ${NCCL_INCLUDE_DIR} # Include NCCL headers
  )
  target_link_libraries(torchfort PRIVATE CUDA::cudart) # Link against CUDA runtime
  target_compile_definitions(torchfort PRIVATE ENABLE_GPU) # Define macro for GPU compilation
endif()

# Link the necessary libraries for the C++ library (from parent).
target_link_libraries(torchfort PRIVATE ${TORCH_LIBRARIES})
target_link_libraries(torchfort PRIVATE ${NCCL_LIBRARY})
target_link_libraries(torchfort PRIVATE MPI::MPI_CXX) # Link against MPI C++ interface
target_link_libraries(torchfort PRIVATE ${YAML_CPP_LIBRARY})

# Compilation options for the C++ library (from parent).
target_compile_definitions(torchfort PRIVATE YAML_CPP_STATIC_DEFINE)
target_compile_options(torchfort PRIVATE $<$<COMPILE_LANGUAGE:CXX>:${TORCH_CXX_FLAGS}>)

# Public headers for the C++ library (copy to src/csrc/include/ relative to this CMakeLists.txt)
set(public_headers
  ${CMAKE_CURRENT_SOURCE_DIR}/src/csrc/include/torchfort.h
  ${CMAKE_CURRENT_SOURCE_DIR}/src/csrc/include/torchfort_rl.h
  ${CMAKE_CURRENT_SOURCE_DIR}/src/csrc/include/torchfort_enums.h
)
set_target_properties("torchfort" PROPERTIES PUBLIC_HEADER "${public_headers}")

# Installation rules for the C++ library (if you wish to install)
install(
  TARGETS torchfort
  EXPORT "torchfortTargets"
  PUBLIC_HEADER DESTINATION ${CMAKE_INSTALL_PREFIX}/include
  INCLUDES DESTINATION ${CMAKE_INSTALL_PREFIX}/include # Install headers to include
  RUNTIME DESTINATION ${CMAKE_INSTALL_PREFIX}/lib # Install the shared library to lib
)


# --- Fortran Library/Module Definition (from parent CMakeLists.txt) ---
# The Fortran examples also link against this Fortran module/library.
add_library(torchfort_fort SHARED) # Keep the original target name.
set_target_properties(torchfort_fort PROPERTIES LIBRARY_OUTPUT_DIRECTORY ${CMAKE_BINARY_DIR}/lib)
# Define where the Fortran compiler should put the module files (.mod).
set_target_properties(torchfort_fort PROPERTIES Fortran_MODULE_DIRECTORY ${CMAKE_BINARY_DIR}/include) # Put .mod in CMAKE_BINARY_DIR/include

# Fortran compilation options for the library (from parent).
if (CMAKE_Fortran_COMPILER_ID STREQUAL "NVHPC")
  # Create the -gpu argument string for GPU compilation with nvfortran (from parent).
  # Depends on TORCHFORT_CUDA_CC_LIST and NVHPC_CUDA_VERSION.
  set(CUF_GPU_ARG "") # Initialize the variable before using APPEND
  foreach(CUDA_CC ${TORCHFORT_CUDA_CC_LIST})
    list(APPEND CUF_GPU_ARG "cc${CUDA_CC}")
  endforeach()
  if (NVHPC_CUDA_VERSION) # Check if variable exists before using
    list(APPEND CUF_GPU_ARG "cuda${NVHPC_CUDA_VERSION}")
  endif()
  list(JOIN CUF_GPU_ARG "," CUF_GPU_ARG)
  message(STATUS "Generated CUF_GPU_ARG: ${CUF_GPU_ARG}") # Show the generated argument

  target_compile_options(torchfort_fort PRIVATE $<$<COMPILE_LANGUAGE:Fortran>:-cpp -cuda -gpu=${CUF_GPU_ARG}>)
elseif (CMAKE_Fortran_COMPILER_ID STREQUAL "GNU")
  target_compile_options(torchfort_fort PRIVATE $<$<COMPILE_LANGUAGE:Fortran>:-cpp>)
endif()

# Test if MPI_Comm_f2c/c2f is available (from parent). Requires test_mpi_f2c.f90 file.
try_compile(
  TEST_F2C_RESULT
  ${CMAKE_BINARY_DIR}
  ${CMAKE_CURRENT_SOURCE_DIR}/cmake/test_mpi_f2c.f90 # Path relative to the new CMakeLists.txt
  LINK_LIBRARIES MPI::MPI_Fortran
)
if (NOT TEST_F2C_RESULT)
  message(STATUS "Could not link MPI_Comm_f2c in Fortran module. Setting -DMPICH flag during module compilation.")
  target_compile_definitions(torchfort_fort PRIVATE MPICH)
endif()

# Fortran source files for the library (copy torchfort_m.F90 to src/fsrc/ relative)
target_sources(torchfort_fort
  PRIVATE
  ${CMAKE_CURRENT_SOURCE_DIR}/src/fsrc/torchfort_m.F90
)
# Link the Fortran library against MPI Fortran (from parent).
target_link_libraries(torchfort_fort PRIVATE MPI::MPI_Fortran)

# Installation rules for the Fortran library and module (if you wish to install)
install(
  TARGETS torchfort_fort
  RUNTIME DESTINATION ${CMAKE_INSTALL_PREFIX}/lib # Install the shared library to lib
)
# Install the Fortran module file (.mod) to the include directory.
install(FILES ${CMAKE_BINARY_DIR}/include/torchfort.mod DESTINATION ${CMAKE_INSTALL_PREFIX}/include)


# --- Fortran Example Executable Definitions (from the first CMakeLists.txt) ---

# Add the 'train' executable target.
add_executable(train)
# Define the source code files (copy to examples/fortran/graph/ relative)
target_sources(train
  PRIVATE
  ${CMAKE_CURRENT_SOURCE_DIR}/examples/fortran/graph/train.f90
  ${CMAKE_CURRENT_SOURCE_DIR}/examples/fortran/graph/simulation.f90 # Assuming this is the copy used
)
# Define the directory for Fortran modules generated by this target.
set_target_properties(train PROPERTIES Fortran_MODULE_DIRECTORY ${CMAKE_BINARY_DIR}/mod/train) # Put modules in a specific build dir


# Add the 'train_distributed' executable target.
add_executable(train_distributed)
# Define the source code files (copy to examples/fortran/simulation/ relative)
target_sources(train_distributed
  PRIVATE
  ${CMAKE_CURRENT_SOURCE_DIR}/examples/fortran/simulation/train_distributed.f90
  ${CMAKE_CURRENT_SOURCE_DIR}/examples/fortran/simulation/simulation.f90 # Assuming this is the copy used
)
# Define the directory for Fortran modules generated by this target.
set_target_properties(train_distributed PROPERTIES Fortran_MODULE_DIRECTORY ${CMAKE_BINARY_DIR}/mod/train_distributed) # Put modules in another specific build dir

# List of executable targets for easier iteration or installation.
set(fortran_example_targets train train_distributed)


# --- Common Configurations for Executables (from the foreach loop in the first CMakeLists.txt) ---
foreach(tgt ${fortran_example_targets})

  # Define the include directories for the current target.
  # Includes where to find generated Fortran modules (our BINAry_DIR/include),
  # MPI Fortran headers, and HDF5 Fortran headers.
  target_include_directories(${tgt}
    PRIVATE
    ${CMAKE_BINARY_DIR}/include # To find torchfort.mod and other generated modules
    ${MPI_Fortran_INCLUDE_DIRS} # MPI headers for Fortran
    ${HDF5_Fortran_INCLUDE_DIRS} # HDF5 headers for Fortran
  )

  # Specify the libraries against which the current target should be linked.
  # Links against MPI Fortran, HDF5 Fortran, the internal Fortran library, and the internal C++ library.
  target_link_libraries(${tgt} PRIVATE MPI::MPI_Fortran)
  target_link_libraries(${tgt} PRIVATE hdf5::hdf5_fortran)
  target_link_libraries(${tgt} PRIVATE torchfort_fort) # Link against the Fortran lib we defined above
  target_link_libraries(${tgt} PRIVATE torchfort) # Link against the C++ lib we defined above

  # Compiler-specific compilation/linking options (from the first CMakeLists.txt).
  if (CMAKE_Fortran_COMPILER_ID STREQUAL "NVHPC")
    # Use the previously generated CUF_GPU_ARG.
    target_compile_options(${tgt} PRIVATE $<$<COMPILE_LANGUAGE:Fortran>:-cpp -acc -gpu=${CUF_GPU_ARG}>)
    target_link_options(${tgt} PRIVATE $<$<COMPILE_LANGUAGE:Fortran>: -acc -gpu=${CUF_GPU_ARG}>)
  elseif (CMAKE_Fortran_COMPILER_ID STREQUAL "GNU")
    target_compile_options(${tgt} PRIVATE $<$<COMPILE_LANGUAGE:Fortran>:-cpp -fbackslash -fopenacc>)
    target_link_options(${tgt} PRIVATE $<$<COMPILE_LANGUAGE:Fortran>: -fopenacc>)
  endif()

endforeach()


# --- Installation Rules ---

# Install the Fortran executables.
install(
  TARGETS ${fortran_example_targets}
  # Define the destination directory relative to the installation prefix.
  RUNTIME DESTINATION ${CMAKE_INSTALL_PREFIX}/bin/examples/fortran/simulation
)

# Install data files and Python scripts required for the examples.
# (copy to examples/fortran/simulation/ relative to this CMakeLists.txt)
install(
  FILES
  ${CMAKE_CURRENT_SOURCE_DIR}/examples/fortran/simulation/config_mlp_native.yaml
  ${CMAKE_CURRENT_SOURCE_DIR}/examples/fortran/simulation/config_fcn_torchscript.yaml
  ${CMAKE_CURRENT_SOURCE_DIR}/examples/fortran/simulation/generate_fcn_model.py
  ${CMAKE_CURRENT_SOURCE_DIR}/examples/fortran/simulation/visualize.py
  # Define the destination directory, the same as the executables.
  DESTINATION ${CMAKE_INSTALL_PREFIX}/bin/examples/fortran/simulation
)

## repository/examples/fortran/simulation

In [ ]:
%%writefile CMakeLists.txt
# Finds the HDF5 package, essential for reading and writing data in HDF5 format.
# We specify that we need the Fortran component of HDF5.
# The REQUIRED option ensures that the CMake configuration will fail if HDF5 (with Fortran support) is not found.
find_package(HDF5 COMPONENTS Fortran REQUIRED)

# Defines a list of Fortran executable targets that will be built.
# In this case, we have two executables: 'train' and 'train_distributed'.
set(fortran_example_targets
  train
  train_distributed
)

# Adds an executable target named 'train'.
add_executable(train)
# Defines the source code files that will be compiled to create the 'train' executable.
# The PRIVATE keyword indicates that these files are specific to the build of this target
# and should not be exposed to other targets that might link against it.
target_sources(train
  PRIVATE
  train.f90
  simulation.f90
)

# Sets specific properties for the 'train' target.
# In this case, we are setting the directory where the Fortran modules generated during compilation
# should be placed. ${CMAKE_CURRENT_SOURCE_DIR} represents the current directory of the CMakeLists.txt file.
# 'mod/0' is the subdirectory within the source directory where the modules will be stored.
set_target_properties(train PROPERTIES Fortran_MODULE_DIRECTORY ${CMAKE_CURRENT_SOURCE_DIR}/mod/0 )

# Adds another executable target named 'train_distributed'.
add_executable(train_distributed)

# Defines the source code files for the 'train_distributed' executable.
target_sources(train_distributed
  PRIVATE
  train_distributed.f90
  simulation.f90
)

# Sets the properties for the 'train_distributed' target, similar to 'train',
# but with a different Fortran module directory ('mod/1'). This can be useful to avoid
# module name conflicts if the two executables have dependencies on modules with the same name.
set_target_properties(train_distributed PROPERTIES Fortran_MODULE_DIRECTORY ${CMAKE_CURRENT_SOURCE_DIR}/mod/1 )

# Starts a loop over each target listed in 'fortran_example_targets' ('train' and 'train_distributed').
foreach(tgt ${fortran_example_targets})

# Defines the include directories for the current target ($tgt).
# PRIVATE means these directories are only needed during the compilation of this target.
# ${CMAKE_BINARY_DIR}/include: Include directory within the CMake build directory.
#                              May contain header files generated during the build process.
# ${MPI_Fortran_INCLUDE_DIRS}: Include directories required to compile Fortran code that uses MPI (Message Passing Interface).
#                               This variable is usually set by the FindMPI.cmake package.
# ${HDF5_Fortran_INCLUDE_DIRS}: Include directories required to compile Fortran code that uses the HDF5 library.
#                                This variable is set by the find_package(HDF5) we found earlier.
  target_include_directories(${tgt}
    PRIVATE
    ${CMAKE_BINARY_DIR}/include
    ${MPI_Fortran_INCLUDE_DIRS}
    ${HDF5_Fortran_INCLUDE_DIRS}
  )

# Specifies the libraries that the current target ($tgt) should link against.
# PRIVATE means these libraries are only needed for this target.
# MPI::MPI_Fortran: Fortran interface of the MPI library (provided by the FindMPI package).
# hdf5::hdf5_fortran: Fortran interface of the HDF5 library (provided by find_package(HDF5)).
# "${PROJECT_NAME}_fort": An internal Fortran library of the project (name is derived from the project name).
# ${PROJECT_NAME}: An internal C/C++ library of the project (name is the project name defined in the 'project(...)' command).
  target_link_libraries(${tgt} PRIVATE MPI::MPI_Fortran)
  target_link_libraries(${tgt} PRIVATE hdf5::hdf5_fortran)
  target_link_libraries(${tgt} PRIVATE "${PROJECT_NAME}_fort")
  target_link_libraries(${tgt} PRIVATE ${PROJECT_NAME})

# Checks which Fortran compiler is being used.
  if (CMAKE_Fortran_COMPILER_ID STREQUAL "NVHPC")

# Defines specific compilation options for the NVHPC (NVIDIA HPC SDK) compiler.
# $<$<COMPILE_LANGUAGE:Fortran>:...> applies options only when the compile language is Fortran.
# -cpp: Enables the C preprocessor for Fortran files.
# -acc: Enables OpenACC directives for parallel programming on GPUs.
# -gpu=${CUF_GPU_ARG}: Passes a specific GPU argument (the CUF_GPU_ARG variable must be defined elsewhere).
    target_compile_options(${tgt} PRIVATE $<$<COMPILE_LANGUAGE:Fortran>:-cpp -acc -gpu=${CUF_GPU_ARG}>)

# Defines specific linking options for the NVHPC compiler.
# The options are similar to the compile options, indicating that linking should also consider GPU acceleration.
    target_link_options(${tgt} PRIVATE $<$<COMPILE_LANGUAGE:Fortran>: -acc -gpu=${CUF_GPU_ARG}>)

# Otherwise, if the Fortran compiler is GNU gfortran.
  elseif (CMAKE_Fortran_COMPILER_ID STREQUAL "GNU")

# Defines specific compilation options for the GNU Fortran compiler.
# -cpp: Enables the C preprocessor for Fortran files.
# -fbackslash: Allows the use of backslashes to continue Fortran code lines (common in older code).
# -fopenacc: Enables OpenACC support for parallel programming.
    target_compile_options(${tgt} PRIVATE $<$<COMPILE_LANGUAGE:Fortran>:-cpp -fbackslash -fopenacc>)

# Defines specific linking options for the GNU Fortran compiler.
# -fopenacc: Ensures that OpenACC libraries are linked.
    target_link_options(${tgt} PRIVATE $<$<COMPILE_LANGUAGE:Fortran>: -fopenacc>)
  endif()

# End of the foreach loop.
endforeach()

# Defines the installation rules for the Fortran executable targets.
install(
  TARGETS ${fortran_example_targets}

# Specifies the destination directory for the executables during installation.
# ${CMAKE_INSTALL_PREFIX} is a user-configurable installation prefix (usually /usr/local or /opt/...).
# The executables will be placed in ${CMAKE_INSTALL_PREFIX}/bin/examples/fortran/simulation.
  RUNTIME DESTINATION ${CMAKE_INSTALL_PREFIX}/bin/examples/fortran/simulation
)

# Defines the installation rules for data files and Python scripts.
install(
  FILES ${CMAKE_CURRENT_SOURCE_DIR}/config_mlp_native.yaml
        ${CMAKE_CURRENT_SOURCE_DIR}/config_fcn_torchscript.yaml
        ${CMAKE_CURRENT_SOURCE_DIR}/generate_fcn_model.py
        ${CMAKE_CURRENT_SOURCE_DIR}/visualize.py

# Specifies the destination directory for these files during installation,
# in the same location as the Fortran executables.
  DESTINATION ${CMAKE_INSTALL_PREFIX}/bin/examples/fortran/simulation)

## Dockerfile.txt

In [ ]:
%%writefile Dockerfile.txt

# Base image: Start from an NVIDIA CUDA development image.
# This provides a base Ubuntu system with CUDA pre-installed, which is essential
# for building a GPU-accelerated library like TorchFort.
FROM nvcr.io/nvidia/cuda:12.3.1-devel-ubuntu22.04

# Install System Dependencies
# Set the DEBIAN_FRONTEND environment variable to noninteractive to prevent
# prompts during package installation.
ENV DEBIAN_FRONTEND noninteractive

# Run an apt update and install core system dependencies.
# curl, unzip, wget are for downloading files.
# cmake is required to build TorchFort.
# python3, python-is-python3, python3-pip are for Python support and package installation.
# python3-pybind11 is specifically needed for the pybind11 C++ binding library.
# git is needed to clone repositories (like NCCL, yaml-cpp, HDF5).
# vim is a text editor (included in the base image for convenience).
# gfortran is the GNU Fortran compiler (needed if not using NVHPC or for some dependencies).
# doxygen is needed for building the documentation (though not built *in* this Dockerfile, it's listed as a dependency).
# libibverbs-dev, ibverbs-utils, numactl are related to InfiniBand/networking, common in HPC environments, likely for MPI.
RUN apt update -y && \
    apt install -y curl unzip wget cmake && \
    apt install -y python3 python-is-python3 python3-pip python3-pybind11 && \
    apt install -y git vim gfortran doxygen && \
    apt install -y libibverbs-dev ibverbs-utils numactl

# Install NVHPC SDK
# Download the NVIDIA HPC SDK installer.
# The SDK includes NVHPC compilers (nvfortran, nvc++) and communication libraries (MPI, NCCL).
RUN wget https://developer.download.nvidia.com/hpc-sdk/24.1/nvhpc_2024_241_Linux_x86_64_cuda_12.3.tar.gz && \
    # Extract the installer.
    tar xpzf nvhpc_2024_241_Linux_x86_64_cuda_12.3.tar.gz && \
    # Run the installer silently.
    nvhpc_2024_241_Linux_x86_64_cuda_12.3/install --quiet && \
    # Clean up the installer files.
    rm -rf nvhpc_2024_241_Linux_x86_64_cuda_12.3 nvhpc_2024_241_Linux_x86_64_cuda_12.3.tar.gz

# Set environment variables to include NVHPC binaries and libraries in the PATH and LD_LIBRARY_PATH.
ENV PATH /opt/nvidia/hpc_sdk/Linux_x86_64/24.1/compilers/bin:$PATH
ENV PATH /opt/nvidia/hpc_sdk/Linux_x86_64/24.1/comm_libs/mpi/bin:$PATH
ENV LD_LIBRARY_PATH /opt/nvidia/hpc_sdk/Linux_x86_64/24.1/cuda/lib64:$LD_LIBRARY_PATH
ENV LD_LIBRARY_PATH /opt/nvidia/hpc_sdk/Linux_x86_64/24.1/comm_libs/mpi/lib:$LD_LIBRARY_PATH
ENV LD_LIBRARY_PATH /opt/nvidia/hpc_sdk/Linux_x86_64/24.1/comm_libs/nvshmem/lib:$LD_LIBRARY_PATH
ENV LD_LIBRARY_PATH /opt/nvidia/hpc_sdk/Linux_x86_64/24.1/math_libs/lib64:$LD_LIBRARY_PATH

# Set CUDA_HOME environment variable.
ENV CUDA_HOME /opt/nvidia/hpc_sdk/Linux_x86_64/24.1/cuda

# Configure HPCx (part of communication libs) initialization in .bashrc for interactive sessions.
RUN echo "source /opt/nvidia/hpc_sdk/Linux_x86_64/24.1/comm_libs/12.3/hpcx/latest/hpcx-init.sh; hpcx_load" >> /root/.bashrc

# Install newer NCCL for compatibility with PyTorch 2.2.1+
# Change directory to /opt.
RUN cd /opt && \
    # Clone the NCCL repository from GitHub, specifying a specific version tag.
    git clone --branch v2.20.3-1 https://github.com/NVIDIA/nccl.git && \
    # Change into the cloned NCCL directory.
    cd nccl && \
    # Build NCCL source, specifying CUDA compute capabilities and CUDA_HOME.
    # The compute capabilities (70, 80, 90) are consistent with the default
    # TORCHFORT_CUDA_CC_LIST in the main CMakeLists.txt.
    make -j src.build NVCC_GENCODE="-gencode=arch=compute_70,code=sm_70 -gencode=arch=compute_80,code=sm_80 -gencode=arch=compute_90,code=sm_90" CUDA_HOME=/opt/nvidia/hpc_sdk/Linux_x86_64/24.1/cuda

# Add the newly built NCCL library path to LD_LIBRARY_PATH.
ENV LD_LIBRARY_PATH /opt/nccl/build/lib:$LD_LIBRARY_PATH

# Install PyTorch
# Use pip to install a specific version of PyTorch. This version is likely compatible
# with the CUDA and NCCL versions installed.
RUN pip3 install torch==2.4.0

# Install yaml-cpp
# Clone the yaml-cpp library from GitHub, specifying a specific version tag.
RUN git clone https://github.com/jbeder/yaml-cpp.git --branch 0.8.0 && \
    # Change into the cloned directory.
    cd yaml-cpp && \
    # Create and enter a build directory.
    mkdir build && cd build && \
    # Configure the build using CMake.
    # Set installation prefix to /opt/yaml-cpp.
    # Set CXX flags, including -D_GLIBCXX_USE_CXX11_ABI=0 for ABI compatibility with PyTorch.
    # Disable building shared libraries (BUILD_SHARED_LIBS=OFF), preferring static libs.
    # Enable position-independent code (CMAKE_POSITION_INDEPENDENT_CODE=ON), good practice for libraries.
    # '..' refers to the parent directory containing the CMakeLists.txt.
    cmake -DCMAKE_INSTALL_PREFIX=/opt/yaml-cpp \
          -DCMAKE_CXX_FLAGS:="-D_GLIBCXX_USE_CXX11_ABI=0" \
          -DBUILD_SHARED_LIBS=OFF \
          -DCMAKE_POSITION_INDEPENDENT_CODE=ON .. && \
    # Build the library using parallel jobs.
    make -j$(nproc) && \
    # Install the built library.
    make install

# Add the installed yaml-cpp library path to LD_LIBRARY_PATH.
ENV LD_LIBRARY_PATH /opt/yaml-cpp/lib:${LD_LIBRARY_PATH}

# Install HDF5
# Download the HDF5 source tarball.
RUN wget https://github.com/HDFGroup/hdf5/archive/refs/tags/hdf5-1_14_3.tar.gz && \
    # Extract the tarball.
    tar xzf hdf5-1_14_3.tar.gz && \
    # Change into the extracted source directory.
    cd hdf5-hdf5-1_14_3 && \
    # Configure the build using the configure script.
    # Set C and Fortran compilers to mpicc and mpifort (MPI wrappers from NVHPC SDK).
    # Set Fortran and C flags to -fPIC for position-independent code.
    # Enable parallel HDF5 support (--enable-parallel).
    # Enable Fortran bindings (--enable-fortran).
    # Set installation prefix to /opt/hdf5.
    CC=mpicc FC=mpifort FCFLAGS=-fPIC CFLAGS=-fPIC \
    ./configure --enable-parallel \
                --enable-fortran \
                --prefix=/opt/hdf5 && \
    # Build and install using parallel jobs.
    make -j$(nproc) install && \
    # Change back to the parent directory.
    cd .. && \
    # Clean up source and tarball files.
    rm -rf hdf5-hdf5-1_14_3 hdf5-1_14_3.tar.gz

# Add the installed HDF5 library path to LD_LIBRARY_PATH.
ENV LD_LIBRARY_PATH /opt/hdf5/lib:$LD_LIBRARY_PATH

# Install additional Python dependencies
# Install Python packages required by TorchFort and its examples/docs using pip.
RUN pip3 install wandb ruamel-yaml h5py matplotlib pygame moviepy

# Install TorchFort
# Set the Fortran compiler environment variable.
ENV FC=nvfortran
# Set the HDF5_ROOT environment variable, used by TorchFort's CMake to find HDF5.
ENV HDF5_ROOT=/opt/hdf5

# Copy the entire TorchFort source code from the build context into the /torchfort directory in the container.
COPY . /torchfort

# Build TorchFort inside the container.
# Change into the /torchfort directory.
RUN cd /torchfort && \
    # Create and enter a build directory.
    mkdir build && cd build && \
    # Run CMake configuration.
    # Set CUDA_PATH environment variable.
    # Set the installation prefix to /opt/torchfort.
    # Explicitly set the C++ compiler to g++ (as nvc++ is not supported for C++ files).
    # Point CMake to the installed yaml-cpp root.
    # Point CMake to the built NCCL root.
    # Enable building examples.
    # Enable building tests.
    # Set CMAKE_PREFIX_PATH to include the PyTorch installation path, found using a Python command.
    # '..' refers to the parent directory containing the root CMakeLists.txt.
    CUDA_PATH=/opt/nvidia/hpc_sdk/Linux_x86_64/24.1/cuda \
    cmake -DCMAKE_INSTALL_PREFIX=/opt/torchfort \
          -DCMAKE_CXX_COMPILER=`which g++` \
          -DTORCHFORT_YAML_CPP_ROOT=/opt/yaml-cpp \
          -DTORCHFORT_NCCL_ROOT=/opt/nccl/build \
          -DTORCHFORT_BUILD_EXAMPLES=1 \
          -DTORCHFORT_BUILD_TESTS=1 \
          -DCMAKE_PREFIX_PATH="`python -c 'import torch;print(torch.utils.cmake_prefix_path)'`" \
          .. && \
    # Build and install TorchFort using parallel jobs.
    make -j$(nproc) install && \
    # Optional: Clean up the build directory after installation (commented out in source).
    #cd / && rm -rf torchfort
    : # This colon is a placeholder for the commented out line above
   

# Add the installed TorchFort library path to LD_LIBRARY_PATH.
ENV LD_LIBRARY_PATH /opt/torchfort/lib:${LD_LIBRARY_PATH}
# Add the PyTorch library path to LD_LIBRARY_PATH. The specific path is for Python 3.10 on Ubuntu 22.04.
ENV LD_LIBRARY_PATH /usr/local/lib/python3.10/dist-packages/torch/lib:${LD_LIBRARY_PATH}

# Set the default command to run when the container starts.
# This starts a bash shell, keeping the container running and allowing interaction.
ENTRYPOINT bash

This Dockerfile defines a multi-stage process within a single file (though it's not explicitly labeled as multi-stage, the process of installing dependencies and then building the main project inside is typical). It starts with a base CUDA image, installs numerous system libraries and tools, downloads and builds core dependencies like NVHPC SDK, NCCL, yaml-cpp, and HDF5, installs PyTorch via pip, and finally copies the TorchFort source code into the container and builds and installs it.

The use of environment variables like `PATH`, `LD_LIBRARY_PATH`, `FC`, and `HDF5_ROOT`, as well as specific CMake flags like `-DCMAKE_CXX_COMPILER`, `-DTORCHFORT_YAML_CPP_ROOT`, `-DTORCHFORT_NCCL_ROOT`, `-DTORCHFORT_BUILD_EXAMPLES`, and `-DTORCHFORT_BUILD_TESTS`, are crucial for correctly configuring the build process within the container environment, ensuring that TorchFort finds all its dependencies and is built with the desired features (like examples and tests) enabled. The Dockerfile also specifies the use of the NVHPC compilers for Fortran (`nvfortran`) and GNU (`g++`) for C++, as required by the main `CMakeLists.txt`.